# Retrieval Augmented Generation - Procurement Contract Analyst -  Palm2 & LangChain


<table align="left">
  <td style="text-align: center">
    <a href="https://colab.research.google.com/github/guruvittal/codesamples/blob/main/Retrieval_Augmented_Generation_-_Contract_Analyst.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Google Colaboratory logo"><br> Run in Colab
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://github.com/guruvittal/codesamples/blob/main/Retrieval_Augmented_Generation_-_Contract_Analyst.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo"><br> View on GitHub
    </a>
  </td>
  <td style="text-align: center">
    <a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/guruvittal/codesamples/blob/main/Retrieval_Augmented_Generation_-_Contract_Analyst.ipynb">
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo"><br> Open in Vertex AI Workbench
    </a>
  </td>
</table>

## Installation & Authentication

**Install google-cloud-aiplatform & langchain**

In [ ]:
# Install langchain and related libraries
!pip install langchain unstructured

# Install Vertex AI LLM SDK
! pip install google-cloud-aiplatform

# Store docs in local vectorstore as index
!pip install -q chromadb


**Authenticate**
Within the colab a simple user authentication is adequate.

In [1]:
import sys

if "google.colab" in sys.modules:
    from google.colab import auth as google_auth
    google_auth.authenticate_user()



**Reference Libraries**
In this section, we will identify all the library classes that will be referenced in the code.



In [7]:
# Chroma DB as Vector Store Database
from langchain.vectorstores import Chroma

# Using Vertex AI
import vertexai
from google.cloud import aiplatform
print(f"Vertex AI SDK version: {aiplatform.__version__}")

# Using Google Cloud Storage Directory loader from langchain
from langchain.document_loaders import GCSDirectoryLoader

import time

from pydantic import BaseModel, Extra, root_validator
from typing import Any, Mapping, Optional, List, Dict
from langchain.llms.base import LLM
from langchain.llms import VertexAI
from langchain.embeddings.base import Embeddings

from langchain.embeddings import VertexAIEmbeddings

Vertex AI SDK version: 1.35.0


## Initiatlize Vertex AI

**We will need a project id and location where the Vertex compute and embedding will be hosted**

In [4]:
PROJECT_ID = "PROJECT_ID"  # @param {type:"string"}

LOCATION = "us-central1"  # @param {type:"string"}

# Initialize Vertex AI SDK
vertexai.init(project=PROJECT_ID, location=LOCATION)

## Ingest the Contracts to build the context for the LLM

*Load all the Procurement Contract Documents*

In [ ]:
loader = GCSDirectoryLoader(project_name=PROJECT_ID, bucket="contractunderstandingatticusdataset")
documents = loader.load()

*Split documents into chunks as needed by the token limit of the LLM and let there be an overlap between the chunks*

In [6]:
# split the documents into chunks
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=100)
docs = text_splitter.split_documents(documents)
print(f"# of documents = {len(docs)}")

# of documents = 2150


## Structuring the ingested documents in a vector space using a Vector Database

*Create an embedding vector engine for all the text in the contract documents that have been ingested*

In [8]:
# Embedding has a max of 600 requests per minute so we are within limits
# https://cloud.google.com/vertex-ai/docs/quotas

REQUESTS_PER_MINUTE = 590

embedding = VertexAIEmbeddings(requests_per_minute=REQUESTS_PER_MINUTE)

embedding

VertexAIEmbeddings(project=None, location='us-central1', request_parallelism=5, max_retries=6, stop=None, model_name='textembedding-gecko', client=<vertexai.preview.language_models._PreviewTextEmbeddingModel object at 0x7bbf680a4cd0>, temperature=0.0, max_output_tokens=128, top_p=0.95, top_k=40, credentials=None, n=1, streaming=False)

*Create a vector store and store the embeddings in the vector store*

In [9]:
contracts_vector_db = Chroma.from_documents(docs, embedding)

## Obtain handle to the retriever

We will use the native retriever provided by Chroma DB to perform similarity search within the contracts document vector store among the different document chunks so as to return that document chunk which has the lowest vectoral "distance" with the incoming user query.

In [10]:
# Expose index to the retriever
retriever = contracts_vector_db.as_retriever(
    search_type="similarity",
    search_kwargs={"k":2})

## Define a Retrieval QA Chain to use retriever

In [11]:
# Create chain to answer questions
from langchain.chains import RetrievalQA

llm = VertexAI(
    model_name='text-bison-32k',
    max_output_tokens=256,
    temperature=0.1,
    top_p=0.8,
    top_k=40,
    verbose=True,
)

# Uses LLM to synthesize results from the search index.
# We use Vertex PaLM Text API for LLM
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=retriever,
    return_source_documents=True)

## Leverage LLM to search from retriever

*Example:*

In [12]:
query = "Who all entered into agreement with Sagebrush?"
result = qa({"query": query})
print(result)

{'query': 'Who all entered into agreement with Sagebrush?', 'result': ' The company, the bank, and the agent entered into the agreement with Sagebrush. ', 'source_documents': [Document(page_content='Each party cooperated and participated in the drafting and preparation of this Agreement and the documents referred to herein, and any and all drafts relating thereto exchanged among the parties shall be deemed the work product of all of the parties and may not be construed against any party by reason of its drafting or preparation. Accordingly, any rule of law or any legal decision that would require interpretation of any ambiguities in this Agreement against any party that drafted or prepared it is of no application and is hereby expressly waived by each of the parties hereto, and any controversy over interpretations of this Agreement shall be decided without regards to events of drafting or preparation.\n\n[Signature Pages Follow]   7\n\nIN WITNESS WHEREOF, each of the parties hereto has

## Build a Front End

In [ ]:
import gradio as gr
import markdown

def chatbot(inputtext):
    result = qa({"query": inputtext})

    return result['result'], get_public_url(result['source_documents'][0].metadata['source']), result['source_documents'][0].metadata['source']

from google.cloud import storage

def get_public_url(uri):
    """Returns the public URL for a file in Google Cloud Storage."""
    # Split the URI into its components
    components = uri.split("/")

    # Get the bucket name
    bucket_name = components[2]

    # Get the file name
    file_name = components[3]

    storage_client = storage.Client()
    bucket = storage_client.bucket(bucket_name)
    blob = bucket.blob(file_name)
    return blob.public_url


print("Launching Gradio")

iface = gr.Interface(fn=chatbot,
                     inputs=[gr.Textbox(label="Query")],
                     examples=["Who are parties to ADMA agreement", "What is the agreement between MICOA & Stratton Cheeseman", "What is the commission % that Stratton Cheeseman will get from MICOA and how much will they get if MICOA's revenues are $100"],
                     title="Contract Analyst",
                     outputs=[gr.Textbox(label="Response"),
                              gr.Textbox(label="URL"),
                              gr.Textbox(label="Cloud Storage URI")])

iface.launch()

